Inventory Control
===

The goal of this computer assignment is to model an inventory control problem with a finite horizon Markov Decision Process (MDP) and derive the optimal solution.

Problem Description
---
Each month, the manager of a warehouse determines current inventory of a single product. Based on this information, he decides whether or not to order additional stock from a supplier. In doing so, he is faced with a tradeoff between the costs associated with ordering and keeping inventory and the lost sales associated with being unable to satisfy a customer demand for the product. The manager's goal is to maximize the profit. Demand for the product is random with a known probability distribution.

To see if MDP is a good model for this problem discription, you need to check if the decisions at the current time affects the future. In this case, it does because the current order will determine how many units is available to sell now and also affects how many units will remain for the future.

Problem Formulation
---
To model the problem as a finite horizon MDP, we need to specify a tuple $(S, A, P, r, H)$. Suppose the decision epoch is $H$ months. Let $s_t$ denote the number of units in the warehouse in the beginning of month $t$. Suppose the capacity of the warehouse is $M$. Thus,
the state space is $S = \{0, 1, 2, \cdots, M\}$. In state $s$, the manager can order at most $M-s$ units. Thus, the set of admissible controls is $A_{s} = \{0, 1, \cdots, M-s\}$. Suppose the demand $D_t$ at month $t$ has a known time-homogeneous probability distribution $p_j = \mathbb{P}(D_t=j), j=0, 1, 2, \cdots$ and the manager orders $a_t$ items. The system dynamics can be represented as $$s_{t+1} = \max \{s_t + a_t - D_t, 0\}.$$ 

The transition probability is then given by
\begin{align}
p(j \mid s, a) =
\begin{cases}
0 &j > s + a \\
p_{s+a-j} & 1\leq j \leq s+a \\
q_{s+a} & j=0
\end{cases}
\end{align}
where $q_u = \sum_{j=u}^\infty p_j$ (Why?). 

It remains to formulate the reward function. We need to account for three different costs/rewards: ordering costs, storage costs, and selling reward. Suppose the ordering cost $O(u)$ consists of a fixed cost and a cost growing with the order size $u$, i.e.,
\begin{align}
O(u) = 
\begin{cases}
K + c(u) & if u > 0 \\
0 & if u = 0
\end{cases}
\end{align}
Let $h(u)$ be a nondecreasing function denoting the storage cost for $u$ units and suppose there is no cost/reward at the last decision epoch. Finally, if the demand is $j$ units and sufficient units are in the warehouse, a selling reward of $f(j)$ units is obtained. Thus, the reward function can be written as
\begin{align}
r_t(s_t, a_t, s_{t+1}) = f(s_t + a_t - s_{t+1}) - h(s_t + a_t) - O(a_t).
\end{align}
It is more convenient to work with $r_t(s_t, a_t)$. To this end we compute $F_t(u)$, the expected value of revenue received in month $t$ if there are $u$ units in the warehouse as
\begin{align}
F_t(u) = \sum_{j=0}^{u-1}f(j)p_j + f(u)q_u
\end{align}
(Why?). Thus, the reward function is
\begin{align}
r_t(s, a) = F(s+a) - h(s+a) - O(a), \qquad t=1, 2, \cdots, H-1
\end{align}
and $r_H = 0$.

Your Job
---
Let $K = 4, c(u) = 2u, h(u) = u, M=3, H=3, f(u) = 8u$ and 
\begin{align}
p_j = 
\begin{cases}
0.25  & j=0 \\
0.5  & j=1 \\
0.25  & j=2
\end{cases}
\end{align}
For your convenience we have hard coded the transition probability and the reward function. Your job is to implement the $\texttt{optimal_policy_and_value}$ function using the dynamic programming algorithm to compute the optimal policy and value function. Please make sure that your code is in the designated area.

In [24]:
from __future__ import division
from __future__ import print_function
import numpy as np

class InventoryControlMDP():
    def __init__(self, H=4):
        self.H = H
        self.S = 4 # M=3 which means there are four states: 0, 1, 2, 3.
        self.terminal_reward = np.zeros(self.S)
        self.r = self._get_reward()
        self.P = self._get_transition()
        
    def _get_reward(self):
        r = [[0,-1,-2,-5],[5,0,-3,-np.inf],[6,-1,-np.inf,-np.inf],[5,-np.inf,-np.inf,-np.inf]]
        
        return np.array(r)
    
    def _get_transition(self):
        
        P = np.zeros((self.S,self.S,self.S)) ####p(s,a,s')
        p_mat = np.array([[1,0,0,0],[3/4,1/4,0,0],[1/4,1/2,1/4,0],[0,1/4,1/2,1/4]])
        
        for i in range(self.S):
            for j in range(self.S-i):
                for k in range(self.S):
                    P[i][j][k] = p_mat[i+j][k]
        return P
    
    def optimal_policy_and_value(self):
        """
        This function should return two numpy arrays denoting
        the optimal policy and value function.
        """
        policy = np.zeros((self.S, self.H-1)) # element (s, h) denotes the optimal policy at state s and time h
        value = np.zeros((self.S, self.H)) # element (s, h) denotes the value function at state s and time h
        ################ Your Code Here ###############
        for h in reversed(range(self.H)):
            if h == self.H-1:
                value[:, h] = self.terminal_reward
            else:
                action_value = self.r + np.dot(self.P, value[:, h+1])
                value[:, h] = np.amax(action_value, axis=1)
                policy[:, h] = np.argmax(action_value, axis=1)
        
        ################ End of Your Code #############
        return policy, value

mdp = InventoryControlMDP(H=4)
policy, value = mdp.optimal_policy_and_value()
print("The optimal policy is")
print(policy)
print('-'*20)
print("The optimal value is")
print(value)    

The optimal policy is
[[3. 2. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
--------------------
The optimal value is
[[ 4.1875  2.      0.      0.    ]
 [ 8.0625  6.25    5.      0.    ]
 [12.125  10.      6.      0.    ]
 [14.1875 10.5     5.      0.    ]]
